## Import datas

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Use pandas to loadinto a DataFrame
# Y1.csv doesn’t have a header so
# add one when loading the file
X1 = pd.read_csv("data/X1.csv")
Y1 = pd.read_csv("data/Y1.csv", header=None, names=['revenue'])


# ENLEVER colonne "Unnamed" du dataset : utilité ? -> Pas listé dans les features du pdf
X = X1.loc[:,X1.columns != "Unnamed: 0"]

# Creer un data training/test splités a partir du X1 (on garde X2 pour les vrais tests) 
x_train, x_test, y_train, y_test = train_test_split(X, Y1, test_size = 0.2, random_state=1)
LinearRegression().fit(x_train,y_train)

,title,img_url,description,ratings,n_votes,is_adult,production_year,runtime,genres,release_year,studio,img_embeddings,text_embeddings
2911,Peter's Friends,https://m.media-amazon.com/images/M/MV5BMjIxOD...,Peter's Friends: Directed by Kenneth Branagh. ...,7.0,10527.0,0,1992,101,"Comedy,Drama,Romance",1992.0,Gold.,"[0.20658691, 0.97558594, 0.9163058, 0.08475555...","[-0.70248544, 0.36827284, 0.9997198, -0.982914..."
1083,The Scoundrel's Wife,https://m.media-amazon.com/images/M/MV5BMjI2MD...,The Scoundrel's Wife: Directed by Glen Pitre. ...,5.9,374.0,0,2002,95,"Drama,Romance",2003.0,MFD,"[0.081257515, 2.152789, 0.26663333, 0.3047054,...","[-0.64765203, 0.39518222, 0.99960977, -0.97800..."
1078,K-19: The Widowmaker,https://m.media-amazon.com/images/M/MV5BMWY2Yj...,K-19: The Widowmaker: Directed by Kathryn Bige...,6.7,60005.0,0,2002,138,"Drama,History,Thriller",2002.0,Par.,"[0.5244218, 3.1182053, 0.34893748, 0.5167997, ...","[-0.64069104, 0.34661368, 0.9995001, -0.971184..."
2378,The Last Ride,https://m.media-amazon.com/images/M/MV5BMTg4ND...,The Last Ride: Directed by Michael Levine. Wit...,5.7,9.0,0,1997,45,"Action,Crime,Drama",2011.0,C1,"[0.2446876, 0.38175932, 1.1183136, 0.008602032...","[-0.5728717, 0.34366137, 0.9994496, -0.973677,..."
1332,Boarding Gate,https://m.media-amazon.com/images/M/MV5BYjQ1NW...,Boarding Gate: Directed by Olivier Assayas. Wi...,5.1,3555.0,0,2007,106,"Drama,Thriller",2008.0,Magn.,"[0.27959552, 1.6733762, 0.99241996, 0.38898322...","[-0.5550897, 0.29739532, 0.9993638, -0.9802733..."
3161,Sorority Boys,https://m.media-amazon.com/images/M/MV5BMTA1OD...,Sorority Boys: Directed by Wallace Wolodarsky....,5.5,13823.0,0,2002,93,Comedy,2002.0,BV,"[0.29491398, 0.8896925, 1.6464747, 0.33184955,...","[-0.72513545, 0.44726473, 0.99983776, -0.98608..."
2124,Raiders of the Lost Ark,https://m.media-amazon.com/images/M/MV5BZjA3Mz...,Raiders of the Lost Ark: With Christopher Fred...,6.4,69.0,0,1981,\N,"Adventure,Family",1981.0,Par.,"[0.56511915, 3.015322, 0.70752704, 0.14187717,...","[-0.7759832, 0.51055866, 0.99987715, -0.989690..."
1459,Raising Victor Vargas,https://m.media-amazon.com/images/M/MV5BMTY1OD...,Raising Victor Vargas: Directed by Peter Solle...,7.2,5261.0,0,2002,88,"Drama,Romance",2003.0,IDP,"[0.33136278, 1.3979486, 1.0132583, 0.62732106,...","[-0.60919464, 0.39039004, 0.9995504, -0.978248..."
1295,Father Figures,https://m.media-amazon.com/images/M/MV5BNTI0Nz...,Father Figures: Directed by Mark Rosner. With ...,8.1,230.0,0,1994,48,"Comedy,Drama,Romance",2017.0,WB,"[0.19871739, 0.10659703, 0.50839514, 0.2867616...","[-0.6709207, 0.356502, 0.9995416, -0.9767028, ..."
1701,Foolproof,https://m.media-amazon.com/images/M/MV5BMzg4OW...,Foolproof: Directed by William Phillips. With ...,6.5,9830.0,0,2003,94,"Action,Comedy,Crime",2003.0,Ode.,"[0.6686234, 1.9280224, 0.8537453, 0.16025315, ...","[-0.7670714, 0.44608894, 0.9998855, -0.9889676..."
